# Objectif du script :
- Ajouter les coordonnées GPS à la base DVF cleanée en étape 1

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import time

In [2]:
year = '2021_s1'

In [3]:
# Chargement du jeu de données
df = pd.read_csv('./databases/inter/01B - dvf_sans_gps_iris_' + year + '.csv')
df = df.drop(columns = {'Unnamed: 0'})
df = df.rename(columns = {'Commune2': 'Commune Alt'})
df.head()


C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18,20,22,24,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,Surface terrain,Code commune INSEE,Ville,Adresse,Commune Alt,Prix m2,Paris,dep_name,region_name,Mer
0,1,05/01/2021,Vente,1480000.0,4.0,NaN,RUE,0960,DE LA BIENFAISANCE,75008.0,...,NaN,75108,75008 PARIS 08,4 RUE DE LA BIENFAISANCE 75008 PARIS,PARIS,13333.333333,1,Paris,Île-de-France,0
1,1,08/01/2021,Vente,410000.0,111.0,NaN,RUE,8525,SAINT-DENIS,75001.0,...,NaN,75101,75001 PARIS 01,111 RUE SAINT-DENIS 75001 PARIS,PARIS,13225.806452,1,Paris,Île-de-France,0
2,1,07/01/2021,Vente,300000.0,313.0,B,RUE,9382,DES TOURNELLES,75003.0,...,NaN,75103,75003 PARIS 03,313 RUE DES TOURNELLES 75003 PARIS,PARIS,13043.478261,1,Paris,Île-de-France,0
3,1,06/01/2021,Vente,1000000.0,16.0,NaN,RUE,7061,DU PARC ROYAL,75003.0,...,NaN,75103,75003 PARIS 03,16 RUE DU PARC ROYAL 75003 PARIS,PARIS,16666.666667,1,Paris,Île-de-France,0
4,1,08/01/2021,Vente,1525559.0,22.0,NaN,RUE,4897,JEAN GOUJON,75008.0,...,NaN,75108,75008 PARIS 08,22 RUE JEAN GOUJON 75008 PARIS,PARIS,16764.384615,1,Paris,Île-de-France,0


In [4]:
df.shape

(246529, 45)

## Import du fichier adresses avec gps et iris

In [5]:
adresses = pd.read_csv('./databases/inter/02 - export_gps_iris_ALL_' + year + '.csv', dtype = {'code_iris_clean': str,
                                                                                             'Code departement' : str,
                                                                                             'Code postal 5 chiffres' : str,
                                                                                             'Code commune INSEE': str})
adresses.shape

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(353500, 17)

In [6]:
# On supprime les duplicates éventuels
adresses.drop_duplicates()
adresses.shape

(353500, 17)

In [7]:
df.columns

Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Code commune INSEE',
       'Ville', 'Adresse', 'Commune Alt', 'Prix m2', 'Paris', 'dep_name',
       'region_name', 'Mer'],
      dtype='object')

In [8]:
adresses.columns

Index(['Unnamed: 0', 'No voie', 'Type de voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune INSEE', 'Ville', 'Adresse',
       'year', 'INSEE_COM', 'Commune_Lat', 'Commune_Lon', 'lon', 'lat',
       'Code postal 5 chiffres'],
      dtype='object')

In [9]:
adresses = adresses.drop(columns = {'Code departement', 'Code commune INSEE'})
adresses.head()

,Unnamed: 0,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse,year,INSEE_COM,Commune_Lat,Commune_Lon,lon,lat,Code postal 5 chiffres
0,0,5080.0,CHE,DE VOGELAS,1370.0,VAL-REVERMONT,1370 VAL REVERMONT,5080 CHE DE VOGELAS 1370 VAL-REVERMONT,2021,NaN,NaN,NaN,5.385863,46.326941,01370
1,1,NaN,NaN,ROUGEMONT,1290.0,BEY,1290 BEY,ROUGEMONT 1290 BEY,2021,1042.0,46.219175,4.846427,4.843974,46.224228,01290
2,2,7.0,ALL,DES ECUREUILS,1310.0,BUELLAS,1310 BUELLAS,7 ALL DES ECUREUILS 1310 BUELLAS,2021,1065.0,46.210414,5.144875,5.157541,46.201078,01310
3,3,87.0,RTE,DE CERTINES,1250.0,MONTAGNAT,1250 MONTAGNAT,87 RTE DE CERTINES 1250 MONTAGNAT,2021,1254.0,46.168697,5.279621,5.272971,46.156382,01250
4,4,NaN,NaN,COMBET MAILLET,1250.0,CORVEISSIAT,1250 CORVEISSIAT,COMBET MAILLET 1250 CORVEISSIAT,2021,1125.0,46.250511,5.471525,NaN,NaN,01250


In [12]:
df = df.merge(adresses[['Adresse', 'lon', 'lat']], on = "Adresse", how = "left")
df.head()

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,Ville,Adresse,Commune Alt,Prix m2,Paris,dep_name,region_name,Mer,lon,lat
0,1,05/01/2021,Vente,1480000.0,4.0,NaN,RUE,0960,DE LA BIENFAISANCE,75008.0,...,75008 PARIS 08,4 RUE DE LA BIENFAISANCE 75008 PARIS,PARIS,13333.333333,1,Paris,Île-de-France,0,2.321068,48.876849
1,1,08/01/2021,Vente,410000.0,111.0,NaN,RUE,8525,SAINT-DENIS,75001.0,...,75001 PARIS 01,111 RUE SAINT-DENIS 75001 PARIS,PARIS,13225.806452,1,Paris,Île-de-France,0,2.347447,48.858758
2,1,07/01/2021,Vente,300000.0,313.0,B,RUE,9382,DES TOURNELLES,75003.0,...,75003 PARIS 03,313 RUE DES TOURNELLES 75003 PARIS,PARIS,13043.478261,1,Paris,Île-de-France,0,2.367347,48.857758
3,1,06/01/2021,Vente,1000000.0,16.0,NaN,RUE,7061,DU PARC ROYAL,75003.0,...,75003 PARIS 03,16 RUE DU PARC ROYAL 75003 PARIS,PARIS,16666.666667,1,Paris,Île-de-France,0,NaN,NaN
4,1,08/01/2021,Vente,1525559.0,22.0,NaN,RUE,4897,JEAN GOUJON,75008.0,...,75008 PARIS 08,22 RUE JEAN GOUJON 75008 PARIS,PARIS,16764.384615,1,Paris,Île-de-France,0,NaN,NaN


In [13]:
df.columns

Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Code commune INSEE',
       'Ville', 'Adresse', 'Commune Alt', 'Prix m2', 'Paris', 'dep_name',
       'region_name', 'Mer', 'lon', 'lat'],
      dtype='object')

In [14]:
df.to_csv('./databases/inter/02C - dvf_with_gps_iris_' + year + '.csv')